In [43]:
import pandas as pd
import osmnx as ox
import networkx as nx
import folium
import os

In [44]:
# Define the individual places in Bicol Region
places = ["Naga, Bicol Region, 4400, Philippines", "Camarines Sur"]

# Coordinates of Barangay Halls
brgy = [
    {"province": "Camarines Sur", "City": "Naga", "barangay": "Cararayan", "lat": 13.628834653694073, "lon": 123.24006454110972}
]

# Create an empty directed multigraph to combine all places
combined_graph = nx.MultiDiGraph()

In [45]:
# Get path to barangay dataset
file_path = os.path.join('..', 'dataset', 'barangay')

# Save the barangays
df = pd.read_csv(file_path, delimiter="|")

print(df.head())

       Type Island group                   Region Province Municipality  \
0  barangay        Luzon  Bicol Region (Region V)    Albay      Bacacay   
1  barangay        Luzon  Bicol Region (Region V)    Albay      Bacacay   
2  barangay        Luzon  Bicol Region (Region V)    Albay      Bacacay   
3  barangay        Luzon  Bicol Region (Region V)    Albay      Bacacay   
4  barangay        Luzon  Bicol Region (Region V)    Albay      Bacacay   

   Postal code  Population (2020) Philippine major island(s)  \
0         4509               2849                      Luzon   
1         4509               1565                      Luzon   
2         4509               1458                      Luzon   
3         4509                770                      Luzon   
4         4509                180                      Luzon   

                                        Coordinates  \
0  13.2852, 123.7930 (13° 17' North, 123° 48' East)   
1  13.2559, 123.8147 (13° 15' North, 123° 49' East)   

In [46]:
df[['lat', 'lon']] = df['Coordinates'].str.extract(r'([0-9.]+),\s([0-9.]+)')

print(df[['lat', 'lon']].head())

       lat       lon
0  13.2852  123.7930
1  13.2559  123.8147
2  13.2962  123.7938
3  13.2925  123.7887
4  13.2924  123.7910


In [47]:
# Loop through each place and retrieve the graph for each
for place in places:
    graph = ox.graph_from_place(place, custom_filter='["highway"~"motorway|trunk|primary|secondary|tertiary|unclassified|ferry"]', network_type='drive')
    combined_graph = nx.compose(combined_graph, graph)

# Get the centroid of the combined places to initialize the map
centroid = ox.geocode_to_gdf(places[0]).unary_union.centroid
m = folium.Map(location=[centroid.y, centroid.x], zoom_start=8, tiles='OpenStreetMap')

In [48]:
# Get the edges and nodes GeoDataFrames after node removal
gdf_nodes, gdf_edges = ox.graph_to_gdfs(combined_graph, nodes=True, edges=True)

# Add the remaining edges (roads and sea routes) to the map
for _, edge in gdf_edges.iterrows():
    points = [(y, x) for x, y in edge['geometry'].coords]
    folium.PolyLine(points, color="darkred", weight=1.5, opacity=0.6).add_to(m)

# Add the remaining nodes (intersections) to the map
# for _, node in gdf_nodes.iterrows():
#     folium.CircleMarker(location=[node['y'], node['x']],
#                         radius=3, color='blue', fill=True, fill_opacity=0.8).add_to(m)

In [49]:
from folium.plugins import MarkerCluster

marker_cluster = MarkerCluster().add_to(m)

# Add barangays as clustered dispensing points
for _, row in df.iterrows():
    folium.Marker(location=[row['lat'], row['lon']],
                  icon=folium.Icon(color='green', icon='info-sign')).add_to(marker_cluster)

In [50]:
# Create the directory if it doesn't exist
if not os.path.exists("html-outputs"):
    os.makedirs("html-outputs")

# Save the map inside the 'html-outputs' folder
m.save("html-outputs/network-with-dispensing-points.html")